In [3]:
import  numpy as np

In [73]:
from search import Problem

In [520]:
import sys
class EstadoTransporte(object):
    def __init__(self, dimension, ofertas, demandas,\
                      asignaciones=None, costos=None):
        """
            dimension es la dimension de la matriz (debe ser cuadrada)
            ofertas y demandas deben ser listas
        """
        if sum(ofertas) != sum(demandas):
            raise NotImplementedError('Todavia no funciona para desbalances')
        
        self.dimension = dimension
        self.ofertas = np.asanyarray([ofertas]).T
        self.demandas = np.asanyarray([demandas])
        self.total_sin_asignar = sum(self.demandas[0])
        if asignaciones is None:
            self.asignaciones = np.zeros((dimension,dimension))
        else:
            self.asignaciones = asignaciones
        if costos is None:
            self.costos = np.ones((dimension,dimension))
        else:
            self.costos = costos
    def asignar(self, fil,col, cant):
        self.asignaciones[fil,col] = cant
        self.ofertas[fil,0] -= cant
        self.demandas[0,col] -= cant
        self.total_sin_asignar -= cant
    def of_dem_disponibles(self):
        """
        devuelve indicesofertas_disp, indices_demandas_disp
        """
        ixs_ofertas_disp = np.where(~(self.ofertas==0))[0] 
        ixs_demandas_disp = np.where(~(self.demandas==0))[1]
        return ixs_ofertas_disp, ixs_demandas_disp
    def disponibles(self, ret_generator =True):
        f,c = self.of_dem_disponibles()
        if not ret_generator:
            return np.array(np.meshgrid(f,c)).T.reshape(-1,2)
        else: #dudo que yield ayude en algo, pues igual se crea la matriz
            for pos in np.array(np.meshgrid(f,c)).T.reshape(-1,2):
                yield pos
    def acciones_posibles(self):
        f,c = self.of_dem_disponibles() #tendra algo de sentido llamar un generador luego de llamar a esto? 
        print("    Expandiendo nodo...")
        cont = 1
        todas_combinaciones = np.array(np.meshgrid(f,c)).T.reshape(-1,2)
        print("    Todas las combinaciones (shape):",todas_combinaciones.shape)
        print("    Tam. (bytes) de todas las combinaciones (shape):",\
              sys.getsizeof(todas_combinaciones))
        for x,y in np.array(np.meshgrid(f,c)).T.reshape(-1,2):
            print("    Nodos enviados desde este estado:",cont)
            yield x,y,min(self.ofertas[x,0], self.demandas[0,y])
            cont += 1
    @classmethod
    def copiar(cls, obj):
        return cls(obj.demandas.shape[1],\
                   np.copy(obj.ofertas.flatten()),\
                   np.copy(obj.demandas.flatten()),\
                   np.copy(obj.asignaciones))
    
    def __repr__(self):
        s = ''
        s += ' '*len('(array([0]), array') + np.array2string(self.demandas.astype(float),separator=', ') + '\n'
        s += '\n'.join([str(x) for x in zip(self.ofertas,self.asignaciones)])
        return s
    
    def __hash__(self):
        return hash(self.asignaciones.tobytes())
    def __eq__(self, other):
        return self.__class__ == other.__class__ and\
               np.array_equal(self.asignaciones, other.asignaciones)

In [513]:
class Iop2Problem(Problem):
    def actions(self, state):
        for accion in state.acciones_posibles():
            yield accion
    def result(self, state, action):
        nuevo_estado = state.__class__.copiar(state)
        fil, col, cant = action
        nuevo_estado.asignar(fil,col,cant)
        return nuevo_estado
    
    def goal_test(self, state):
        return state.total_sin_asignar <= 0

In [521]:
estado_inicial = EstadoTransporte(4,[4,2,4,5],[2,3,6,4])

In [522]:
problema_iop2 =  Iop2Problem(estado_inicial)

In [516]:
problema_iop2

In [523]:
from search import depth_first_graph_search, Node
from collections import deque

In [524]:
class NodeIter:

    """ El Node original, pero expand devuelve un generador, en vez de una lista
    A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        for action in problem.actions(self.state):
            yield self.child_node(problem, action)

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = NodeIter(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, NodeIter) and self.state == other.state

    def __hash__(self):
        return hash(self.state)


In [525]:
import types
class ValGenList(object):
    """
    Lista que es consciente de que contiene generadores...
    Si se llama a pop y el ultimo elemento es un generador,
    pop() devolvera el primer elemento del generador
    !!!cuidado, no es seguro que prevenga si realmente esta
    vacia
    """
    def __init__(self):
        self._L = []
    def pop(self):
        #raise NotImplementedError("Esta lista debe funcionar como iterador")
        
        if len(self._L) == 0:
            
            raise IndexError("pop from empty list")
        if isinstance(self._L[-1], types.GeneratorType):
            #print("entré a isinstance") #:deb:
            try:
                return next(self._L[-1])
            except StopIteration:
                #print("entré a stopiter") #:deb:
                self._L.pop()
                return self.pop()
        else:
            #print("entré a else") #:deb:
            return self._L.pop()
    def append(self, elem):
        self._L.append(elem)
        return
    def extend(self, other):
        if isinstance(other, types.GeneratorType):
            self._L.append(other)
        else:
            self._L.extend(other)
    
    """def __next__(self):
        if len(self._L) == 0:
            raise StopIteration
        if isinstance(self._L[-1], types.GeneratorType):
            try:
                print("Intentamos llamar al siguiente elemento") #:deb:
                return next(self._L[-1])
            except StopIteration:
                #esto debe manejarse en caso de que el iterador vacio
                #no sea el ultimo elemento
                
                self._L.pop()
                if len(self._L) == 0:
                    raise
                return self.__next__()
            except Exception as e:
                print("Ay :(")
                raise
            
        else:
            try:
                return self._L.pop()
            except:
                raise StopIteration"""
    def __iter__(self):
        return self
    def __contains__(self, key):
        return key in self._L
        
    
    def __bool__(self):
        return bool(self._L)
    def __len__(self):
        return len(self._L)
    def __str__(self):
        return self._L.__str__()
    def __repr__(self):
        return self._L.__repr__()
            
        

In [526]:
import sys
def lazy_depth_first_graph_search(problem, _print=False):
    """ (Utiliza una pila "holgazana", en vez de que la operacion
        expandir devuelva todos los hijos, esta devuelve el hijo
        inmediato junto con un generador para el resto de hijos)
        El problema es en que no verifica si un hijo ya esta en la frontera
        Search the deepest nodes in the search tree first.
        Search through the successors of a problem to find a goal.
        The argument frontier should be an empty stack.
        Does not get trapped by loops.
        If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier = ValGenList()
    frontier.append(NodeIter(problem.initial))  # Stack
    
    explored = set()
    max_tam_frontera = sys.getsizeof(frontier)
    while frontier:
        max_tam_frontera = max(max_tam_frontera, sys.getsizeof(frontier._L))
        print("frontier len:", len(frontier))
        print("frontier size in bytes:",sys.getsizeof(frontier._L))
        print("frontier:",frontier)
        
        try:
            node = frontier.pop()
        except IndexError:
            break
        except StopIteration:
            break
        print("node:", node)
        
        
        
        print()
        if problem.goal_test(node.state):
            print("max_tam_frontera (bytes):", max_tam_frontera)
            print("tam_explorados:", len(explored))
            return node
        explored.add(node.state)
        generador_hijos = (child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
        frontier.extend(generador_hijos)
        
    print("max_tam_frontera (bytes):", max_tam_frontera)
    print("tam_explorados:", len(explored))
    return None

In [527]:
lazy_depth_first_graph_search(problema_iop2)

frontier len: 1
frontier size in bytes: 52
frontier: [<Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>]
node: <Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>

frontier len: 1
frontier size in bytes: 52
frontier: [<generator object lazy_depth_first_graph_search.<locals>.<genexpr> at 0x0EAF8530>]
    Expandiendo nodo...
    Todas las combinaciones (shape): (16, 2)
    Tam. (bytes) de todas las combinaciones (shape): 56
    Nodos enviados desde este estado: 1
node: <Node                   [[0., 3., 6., 4.]]
(array([2]), array([2., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>

frontier len: 2
frontier size in bytes: 5

<Node                   [[0., 0., 0., 0.]]
(array([0]), array([2., 2., 0., 0.]))
(array([0]), array([0., 1., 1., 0.]))
(array([0]), array([0., 0., 4., 0.]))
(array([0]), array([0., 0., 1., 4.]))>

In [536]:
gol3 = lazy_depth_first_graph_search(problema_iop2)

frontier len: 1
frontier size in bytes: 52
frontier: [<Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>]
node: <Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>

frontier len: 1
frontier size in bytes: 52
frontier: [<generator object lazy_depth_first_graph_search.<locals>.<genexpr> at 0x0EAF8530>]
    Expandiendo nodo...
    Todas las combinaciones (shape): (16, 2)
    Tam. (bytes) de todas las combinaciones (shape): 56
    Nodos enviados desde este estado: 1
node: <Node                   [[0., 3., 6., 4.]]
(array([2]), array([2., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>

frontier len: 2
frontier size in bytes: 5

In [510]:
import sys
def depth_first_graph_search_print(problem):
    """Search the deepest nodes in the search tree first.
        Search through the successors of a problem to find a goal.
        The argument frontier should be an empty queue.
        Does not get trapped by loops.
        If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier = [(Node(problem.initial))]  # Stack
    explored = set()
    max_tam_frontera = sys.getsizeof(frontier)
    while frontier:
        print("frontier:",frontier)
        print("frontier len:",len(frontier))
        print("frontier size in bytes:",sys.getsizeof(frontier))
        #max_tam_frontera = max(max_tam_frontera, len(frontier))
        max_tam_frontera = max(max_tam_frontera, sys.getsizeof(frontier))
        node = frontier.pop()
        
        print("node:",node)
        print()
        if problem.goal_test(node.state):
            print("max_tam_frontera (bytes):", max_tam_frontera)
            print("tam_explorados:", len(explored))
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    print("max_tam_frontera (bytes):", max_tam_frontera)
    print("tam_explorados:", len(explored))
    return None

In [480]:

def breadth_first_graph_search_print(problem):
    """[Figure 3.11]
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        print("max_tam_frontera:", 0)
        print("tam_explorados:", 1)
        return node
    frontier = deque([node])
    explored = set()
    max_tam_frontera = 1
    while frontier:
        max_tam_frontera = max(max_tam_frontera, len(frontier))
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    print("max_tam_frontera:", max_tam_frontera)
                    print("tam_explorados:", len(explored))
                    return child
                frontier.append(child)
    print("max_tam_frontera:", max_tam_frontera)
    print("tam_explorados:", len(explored))
    return None


In [535]:
gol = depth_first_graph_search_print(problema_iop2)

frontier: [<Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>]
frontier len: 1
frontier size in bytes: 40
node: <Node                   [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))>

    Expandiendo nodo...
    Todas las combinaciones (shape): (16, 2)
    Tam. (bytes) de todas las combinaciones (shape): 56
    Nodos enviados desde este estado: 1
    Nodos enviados desde este estado: 2
    Nodos enviados desde este estado: 3
    Nodos enviados desde este estado: 4
    Nodos enviados desde este estado: 5
    Nodos enviados desde este estado: 6
    Nodos enviados desde este estado: 7
    Nodos enviados desde este estado: 8
    Nodos enviados desde este estado: 9
    Nodos enviados desde este estado: 10
    Nodos enviados d

(array([0]), array([0., 0., 1., 4.]))>]
frontier len: 44
frontier size in bytes: 268
node: <Node                   [[0., 0., 0., 0.]]
(array([0]), array([2., 2., 0., 0.]))
(array([0]), array([0., 1., 1., 0.]))
(array([0]), array([0., 0., 4., 0.]))
(array([0]), array([0., 0., 1., 4.]))>

max_tam_frontera (bytes): 268
tam_explorados: 7


In [171]:
gol

<Node                   [[0., 0., 0., 0.]]
(array([0]), array([2., 2., 0., 0.]))
(array([0]), array([0., 1., 1., 0.]))
(array([0]), array([0., 0., 4., 0.]))
(array([0]), array([0., 0., 1., 4.]))>

In [172]:
gol2 = breadth_first_graph_search_print(problema_iop2)

max_tam_frontera: 1011
tam_explorados: 728


In [173]:
gol2

<Node                   [[0., 0., 0., 0.]]
(array([0]), array([0., 3., 1., 0.]))
(array([0]), array([2., 0., 0., 0.]))
(array([0]), array([0., 0., 0., 4.]))
(array([0]), array([0., 0., 5., 0.]))>

In [178]:
sol = gol.solution()[::-1]
sol

[(0, 0, 2), (0, 1, 2), (1, 1, 1), (1, 2, 1), (2, 2, 4), (3, 2, 1), (3, 3, 4)]

In [174]:
sol  = gol2.solution()[::-1]
sol

[(3, 2, 5), (2, 3, 4), (1, 0, 2), (0, 2, 1), (0, 1, 3)]

In [539]:
sol  = gol3.solution()
sol

[(0, 0, 2), (0, 1, 2), (1, 1, 1), (1, 2, 1), (2, 2, 4), (3, 2, 1), (3, 3, 4)]

In [540]:
estado_actual = problema_iop2.initial
print(estado_actual)
print()
for accion in sol:
    print("accion:",accion)
    estado_siguiente = problema_iop2.result(estado_actual,accion)
    estado_actual = estado_siguiente
    print(estado_actual)
    print()

                  [[2., 3., 6., 4.]]
(array([4]), array([0., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))

accion: (0, 0, 2)
                  [[0., 3., 6., 4.]]
(array([2]), array([2., 0., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))

accion: (0, 1, 2)
                  [[0., 1., 6., 4.]]
(array([0]), array([2., 2., 0., 0.]))
(array([2]), array([0., 0., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))

accion: (1, 1, 1)
                  [[0., 0., 6., 4.]]
(array([0]), array([2., 2., 0., 0.]))
(array([1]), array([0., 1., 0., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), array([0., 0., 0., 0.]))

accion: (1, 2, 1)
                  [[0., 0., 5., 4.]]
(array([0]), array([2., 2., 0., 0.]))
(array([0]), array([0., 1., 1., 0.]))
(array([4]), array([0., 0., 0., 0.]))
(array([5]), arra